In [1]:
import pandas as pd
import numpy as np
import copy

from datetime import datetime, timedelta, time

In [2]:
from solver import Solver

In [3]:
slvr = Solver('data/flights.json', 'data/crew.json')

slvr.flights

c:\Users\user\ML\Optimisation\crew-assignment\solver.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.flights['dep'] = pd.to_datetime(self.flights['dep']).dt.time
c:\Users\user\ML\Optimisation\crew-assignment\solver.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.flights['arr'] = pd.to_datetime(self.flights['arr']).dt.time


,day,id,origin,dest,dep,arr,duration,type
0,1,D1_F026,FRA,CPH,06:00:00,07:00:00,1,B737
1,1,D1_F043,DUB,CDG,06:00:00,07:00:00,1,B737
2,1,D1_F005,DUB,BCN,06:00:00,08:00:00,2,B737
3,1,D1_F023,FRA,CDG,06:00:00,08:00:00,2,B737
4,1,D1_F031,BCN,MXP,06:00:00,08:00:00,2,A320
...,...,...,...,...,...,...,...,...
245,5,D5_F042,LHR,DUB,18:00:00,19:00:00,1,A320
246,5,D5_F003,DUB,CPH,18:00:00,20:00:00,2,A320
247,5,D5_F001,MXP,CDG,19:00:00,20:00:00,1,A319
248,5,D5_F021,LHR,AMS,19:00:00,20:00:00,1,A320


In [4]:
slvr.crew

,id,name,role,base,qualified
0,C0,Joe Patterson,captain,AMS,"[B737, A320]"
1,C1,Kellie Lucas,captain,DUB,[B737]
2,C2,Joshua Green,captain,AMS,"[B737, A319]"
3,C3,Jeanette Novak,captain,MXP,"[B737, A320]"
4,C4,Amanda Mcdaniel,captain,AMS,[A320]
...,...,...,...,...,...
95,FO45,Jesse Williams,first_officer,MXP,[A319]
96,FO46,Jennifer Kelly,first_officer,DUB,[B737]
97,FO47,Jill Richmond,first_officer,LHR,[A320]
98,FO48,Antonio Gordon,first_officer,CPH,"[A320, A319]"


In [5]:
slvr.initial_assignment()

slvr.assignment

{'D1_F026': {'captain': 'C26', 'first_officer': 'FO20', 'dead_heading': []},
 'D1_F043': {'captain': 'C1', 'first_officer': 'FO2', 'dead_heading': []},
 'D1_F005': {'captain': 'C44', 'first_officer': 'FO46', 'dead_heading': []},
 'D1_F023': {'captain': 'C22', 'first_officer': 'FO20', 'dead_heading': []},
 'D1_F031': {'captain': 'C32', 'first_officer': 'FO32', 'dead_heading': []},
 'D1_F014': {'captain': 'C14', 'first_officer': 'FO43', 'dead_heading': []},
 'D1_F044': {'captain': 'C31', 'first_officer': 'FO15', 'dead_heading': []},
 'D1_F047': {'captain': 'C12', 'first_officer': 'FO1', 'dead_heading': []},
 'D1_F011': {'captain': 'C36', 'first_officer': 'FO2', 'dead_heading': []},
 'D1_F015': {'captain': None, 'first_officer': 'FO6', 'dead_heading': []},
 'D1_F024': {'captain': 'C27', 'first_officer': 'FO30', 'dead_heading': []},
 'D1_F032': {'captain': 'C45', 'first_officer': 'FO31', 'dead_heading': []},
 'D1_F048': {'captain': 'C33', 'first_officer': 'FO25', 'dead_heading': []},
 'D1_

In [6]:
slvr.initial_crew_state()
slvr.crew_state 

{'C0': [{'day': 2,
   'flight': 'D2_F013',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'MXP',
   'depart': datetime.time(16, 0),
   'arrive': datetime.time(17, 0)},
  {'day': 5,
   'flight': 'D5_F045',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'MXP',
   'depart': datetime.time(13, 0),
   'arrive': datetime.time(14, 0)}],
 'C1': [{'day': 1,
   'flight': 'D1_F043',
   'role': 'captain',
   'origin': 'DUB',
   'destination': 'CDG',
   'depart': datetime.time(6, 0),
   'arrive': datetime.time(7, 0)},
  {'day': 1,
   'flight': 'D1_F010',
   'role': 'captain',
   'origin': 'DUB',
   'destination': 'AMS',
   'depart': datetime.time(15, 0),
   'arrive': datetime.time(16, 0)},
  {'day': 2,
   'flight': 'D2_F012',
   'role': 'captain',
   'origin': 'DUB',
   'destination': 'LHR',
   'depart': datetime.time(19, 0),
   'arrive': datetime.time(20, 0)},
  {'day': 3,
   'flight': 'D3_F020',
   'role': 'captain',
   'origin': 'DUB',
   'destination': 'MXP',
   'depart

In [7]:
total_assignment_cost, diagnostics = slvr.compute_assignment_cost(slvr.assignment)
print(total_assignment_cost)

41400


In [8]:
total_crew_cost, diagnostics = slvr.compute_crew_cost(slvr.crew_state)
print(total_crew_cost)

521500


In [9]:
destroy_repair_strength = 0.2
num_destroy_operators = 2
num_repair_operators = 1

num_to_destroy = int(len(slvr.flights) * destroy_repair_strength)

In [10]:
#print(f"{total_assignment_cost}, {total_crew_cost} => {total_assignment_cost+total_crew_cost}, \n {r_assignment_cost}, {r_crew_cost} => {r_assignment_cost+r_crew_cost}")

In [11]:
total_assignment_cost, _ = slvr.compute_assignment_cost(slvr.assignment)
total_crew_cost, _ = slvr.compute_crew_cost(slvr.crew_state)

for i in range(1000):

    d_assignment, d_crew_state, flights_destroyed, affected_crew_list = slvr.destroy(num_to_destroy)

    r_assignment, r_crew_state = slvr.repair()
    r_assignment_cost, _ = slvr.compute_assignment_cost(slvr.r_assignment)
    r_crew_cost, _ = slvr.compute_crew_cost(slvr.r_crew_state)

    if (r_assignment_cost+r_crew_cost) < (total_assignment_cost+total_crew_cost):
        slvr.accept_change()
        total_assignment_cost = r_assignment_cost
        total_crew_cost = r_crew_cost
    
        print(f"{total_assignment_cost}, {total_crew_cost} => {total_assignment_cost+total_crew_cost}")

print("finished")
print(f"{total_assignment_cost}, {total_crew_cost} => {total_assignment_cost+total_crew_cost}")

66900, 493300 => 560200
95300, 445000 => 540300
96300, 434900 => 531200
finished
96300, 434900 => 531200


In [12]:
slvr.assignment

{'D1_F026': {'captain': 'C26', 'first_officer': 'FO8', 'dead_heading': []},
 'D1_F043': {'captain': 'C1', 'first_officer': 'FO2', 'dead_heading': []},
 'D1_F005': {'captain': 'C44', 'first_officer': 'FO46', 'dead_heading': []},
 'D1_F023': {'captain': 'C22', 'first_officer': 'FO8', 'dead_heading': []},
 'D1_F031': {'captain': 'C32', 'first_officer': 'FO16', 'dead_heading': []},
 'D1_F014': {'captain': 'C14', 'first_officer': 'FO43', 'dead_heading': []},
 'D1_F044': {'captain': 'C2', 'first_officer': 'FO15', 'dead_heading': []},
 'D1_F047': {'captain': 'C12', 'first_officer': 'FO1', 'dead_heading': []},
 'D1_F011': {'captain': 'C34', 'first_officer': None, 'dead_heading': []},
 'D1_F015': {'captain': None, 'first_officer': 'FO6', 'dead_heading': []},
 'D1_F024': {'captain': 'C27', 'first_officer': 'FO30', 'dead_heading': []},
 'D1_F032': {'captain': 'C45', 'first_officer': 'FO31', 'dead_heading': []},
 'D1_F048': {'captain': 'C33', 'first_officer': 'FO25', 'dead_heading': []},
 'D1_F000

In [13]:
slvr.crew_state

{'C0': [{'day': 1,
   'flight': 'D1_F045',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'CPH',
   'depart': datetime.time(18, 0),
   'arrive': datetime.time(20, 0),
   'duration': 2},
  {'day': 2,
   'flight': 'D2_F017',
   'role': 'captain',
   'origin': 'CPH',
   'destination': 'CDG',
   'depart': datetime.time(9, 0),
   'arrive': datetime.time(10, 0),
   'duration': 1},
  {'day': 2,
   'flight': 'D2_F026',
   'role': 'captain',
   'origin': 'CPH',
   'destination': 'AMS',
   'depart': datetime.time(9, 0),
   'arrive': datetime.time(10, 0),
   'duration': 1},
  {'day': 2,
   'flight': 'D2_F013',
   'role': 'captain',
   'origin': 'AMS',
   'destination': 'MXP',
   'depart': datetime.time(16, 0),
   'arrive': datetime.time(17, 0)},
  {'day': 3,
   'flight': 'D3_F034',
   'role': 'captain',
   'origin': 'MXP',
   'destination': 'CDG',
   'depart': datetime.time(15, 0),
   'arrive': datetime.time(16, 0),
   'duration': 1},
  {'day': 4,
   'flight': 'D4_F023',
   'role': '